In [ ]:
import tensorflow as tf
#import tensorflow.tools

from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, IntegerType
from kafka import KafkaProducer

import io
import random
import avro.schema
from avro.io import DatumWriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json

# Kafka

In [ ]:
# To send messages synchronously
PRODUCER = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: v)

# Kafka topic
TOPIC = "my-topic"

# Path to user.avsc avro schema
SCHEMA = {
            "type": "record",
            "name": "MultiImageRecord",
            "fields": [
                {"name": "id", "type": "int"},
                {"name": "img1", "type": "bytes"},
                {"name": "img2", "type": "bytes"},
                {"name": "img3", "type": "bytes"}
            ]
        }

for i in range(10):
    writer = DatumWriter(SCHEMA)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer.write({"name": "123", "favorite_color": "111", "favorite_number": random.randint(0, 10)}, encoder)
    raw_bytes = bytes_writer.getvalue()
    PRODUCER.send_messages(TOPIC, raw_bytes)

In [ ]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
i = 0
while i < 100:
    producer.send("Imagen", )
    time.sleep(1)
    i += 1

producer.flush()

In [ ]:
"""
-Saved_model_path es una variable para ingresa la ruta del modelo
-filename es la variable que tendra el nombre del archivo con la red ya entrenado
"""
spark = SparkSession.builder \
    .appName("Clasificacion_imagenes") \
    .getOrCreate()
    
esquema = StructType() \
    .add("id", IntegerType()) \
    .add("value", IntegerType())

spark.sparkContext.addFile("saved_model_path")

def predecir(data):
    model_file_local = SparkFiles.get("filename")
    model = tf.keras.models.load_model(model_file_local, compile=False)
    model.predict(data)

df_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "Imagen") \
    .option("startingOffsets", "earliest") \
    .load()

df = df_raw.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), esquema).alias("data")) \
    .select("data.*")

query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()